In [1]:
#!/usr/bin/python3
import math

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML, display
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    ControllabilityMatrix,
    DiagramBuilder,
    Linearize,
    LinearQuadraticRegulator,
    MeshcatVisualizer,
    Parser,
    Saturation,
    SceneGraph,
    Simulator,
    StartMeshcat,
    WrapToSystem,
)
from pydrake.examples import (
    AcrobotGeometry,
    AcrobotInput,
    AcrobotPlant,
    AcrobotState,
    QuadrotorGeometry,
    QuadrotorPlant,
    StabilizingLQRController,
)
from pydrake.symbolic import Variable
from pydrake.systems.primitives import SymbolicVectorSystem
from pydrake.solvers import MathematicalProgram, Solve

import quadrotor_params as p

from trajectory_generator import TrajectoryGenerator

In [2]:
waypoints = np.array([
    [0.0, 0.0, 0.0],
    [5.0, 2.0, 1.0],
    [8.0, 5.0, 2.0],
    [10.0, 10.0, 3.0],
    [8.0, 15.0, 4.0],
    [5.0, 18.0, 5.0],
    [0.0, 20.0, 6.0],
    [-5.0, 18.0, 7.0],
    [-8.0, 15.0, 8.0],
    [-10.0, 9.0, 9.0],
    [-9.0, 2.0, 10.0],
    [-8.0, -5.0, 11.0],
    [-7.0, -14.0, 11.0],
    [-6.0, -24.0, 10.0],
    [-5.0, -32.0, 9.0],
    [-4.0, -38.0, 8.0],
    [-3.0, -42.0, 7.0],
    [-2.0, -44.0, 6.0],
]).T

headings = np.linspace(0.0, 2*np.pi, 2*waypoints.shape[1]-3).tolist()
for i in range(waypoints.shape[1]-3):
    del headings[-3 - i]
headings = np.array(headings)

traj_gen = TrajectoryGenerator(waypoints, headings)
traj_gen.solve()

# t = np.linspace(0, traj_gen.tf, 1000)
dt = .01
t = np.arange(0.0, traj_gen.tf, dt)

In [3]:
running_as_notebook = True
meshcat = StartMeshcat()
builder = DiagramBuilder()

plant = builder.AddSystem(QuadrotorPlant(
    m_arg=p.m_kg,
    L_arg=p.L_m,
    I_arg=p.J,
    kF_arg=p.kF,
    kM_arg=p.kM
))

# props = [Variable(f'p{i}') for i in range(4)]
# controller = builder.AddSystem(SymbolicVectorSystem(state=props, dynamics=np.zeros(4), output=props))
# builder.Connect(controller.get_output_port(0), plant.get_input_port(0))

# Set up visualization in MeshCat
scene_graph = builder.AddSystem(SceneGraph())
QuadrotorGeometry.AddToBuilder(
    builder, plant.get_output_port(0), scene_graph
)
meshcat.Delete()
meshcat.ResetRenderMode()
meshcat.SetProperty("/Background", "visible", False)
MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)
# end setup for visualization

diagram = builder.Build()

# Set up a simulator to run this diagram
simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0 if running_as_notebook else 0.0)
context = simulator.get_mutable_context()
# print(context.GetInputPort())

# Simulate
for i in range(5):
    context.SetTime(0.0)
    # controller_context = diagram.GetMutableSubsystemContext(controller, context)
    # controller_context.get_mutable_continuous_state_vector().SetFromVector(np.array([0.25, 0.25, 0.25, 0.25])*9.8)
    # context.SetContinuousState(
    #     0.5
    #     * np.random.randn(
    #         12,
    #     )
    # )
    simulator.Initialize()
    
    
    for ti in t:
        simulator.AdvanceTo(ti)
        context.SetContinuousState(
            np.concatenate([traj_gen.eval_trajectory(ti, 0), np.zeros(2), traj_gen.eval_heading(ti, 0), np.zeros(6)])
        )

INFO:drake:Meshcat listening for connections at http://localhost:7000


KeyboardInterrupt: 